In [57]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
import string
import nltk
#nltk.download('stopwords')
nltk.download('wordnet')
pd.set_option('display.max_columns', None)
df = pd.read_csv('labeled_data.csv')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Preprocessing


In [58]:
#check null values
df.isnull().sum() 

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [59]:
stopwords = nltk.corpus.stopwords.words("english")
#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
other_exclusions = ["#ff", "ff", "rt" , "RT"]
stopwords.extend(other_exclusions)
lemmatizer = WordNetLemmatizer()
def preprocess(tweet):  
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweet_space = tweet.str.replace(regex_pat, ' ' , regex=True)
    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(regex_pat, '' , regex=True)
    # removal of links[https://abc.com]
    giant_url_regex =  re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            r'[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace(giant_url_regex, '', regex=True)
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace(r"[^a-zA-Z]", " ", regex=True)
    # remove whitespace with a single space
    newtweet=punc_remove.str.replace(r'\s+', ' ', regex=True)
    # remove leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','', regex=True)
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr', regex=True)
    # removal of capitalization
    tweet_lower = newtweet.str.lower()
    # tokenizing
    tokenized_tweet = tweet_lower.apply(lambda x: x.split())
    # removal of stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [item for item in x if item not in stopwords])
    # stemming of the tweets
   # tokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tweets_p= tokenized_tweet
    return tweets_p
processed_tweets = preprocess(df.tweet)   
df['processed_tweets'] = processed_tweets
print(df[["tweet","processed_tweets"]].head(10))

                                               tweet  \
0  !!! RT @mayasolovely: As a woman you shouldn't...   
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   
5  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...   
6  !!!!!!"@__BrighterDays: I can not just sit up ...   
7  !!!!&#8220;@selfiequeenbri: cause I'm tired of...   
8  " &amp; you might not get ya bitch back &amp; ...   
9  " @rhythmixx_ :hobbies include: fighting Maria...   

                                    processed_tweets  
0  woman complain cleaning house amp man always t...  
1  boy dats cold tyga dwn bad cuffin dat hoe st p...  
2       dawg ever fuck bitch start cry confused shit  
3                                   look like tranny  
4     shit hear might true might faker bitch told ya  
5  shit blows claim faithful somebody still fucki... 

In [60]:
# Assume 'df' is your DataFrame with the dataset
X = df[['processed_tweets']]
y = df['class']

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)



# Convert text to numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer( max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['processed_tweets'])
X_val_tfidf = tfidf_vectorizer.transform(X_val['processed_tweets'])

X_train_final = X_train_tfidf
X_val_final = X_val_tfidf
# Train a model (e.g., RandomForestClassifier)
clf =  LogisticRegression(C = 4, random_state= 42)
clf.fit(X_train_final, y_train)

# Make predictions
predictions = clf.predict(X_val_final)

# Evaluate the model
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy}")
print(classification_report(y_val, predictions))

Accuracy: 0.8958307861596773
              precision    recall  f1-score   support

           0       0.46      0.21      0.29       465
           1       0.92      0.96      0.94      6335
           2       0.85      0.84      0.84      1379

    accuracy                           0.90      8179
   macro avg       0.74      0.67      0.69      8179
weighted avg       0.88      0.90      0.89      8179



C:\Users\youss\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
""" #find best c parameter with cross validation
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(LogisticRegression(random_state=42 , max_iter= 10000), param_grid, cv=5 )
grid_search.fit(X_train_final, y_train)

best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}") """

' #find best hyperparameters for logistic\n# Make predictions for the new sentence\nfrom sklearn.model_selection import GridSearchCV\nparam_grid = {\'C\': [0.001, 0.01, 0.1, 1, 10, 100]}\ngrid_search = GridSearchCV(LogisticRegression(random_state=42 , max_iter= 10000), param_grid, cv=5 )\ngrid_search.fit(X_train_final, y_train)\n\nbest_params = grid_search.best_params_\nprint(f"Best Hyperparameters: {best_params}") '

In [63]:
# Example sentence
new_sentence = pd.Series('women')

new_sentence = preprocess(new_sentence)
print(new_sentence)


# Convert the new sentence to TF-IDF features
new_sentence_tfidf = tfidf_vectorizer.transform([new_sentence[0]])

# Make predictions for the new sentence
prediction = clf.predict(new_sentence_tfidf)

# Print the prediction
classifciationType = ''
if(prediction[0] == 0):
    classifciationType = 'Hate Speech'
elif(prediction[0] == 1): 
        classifciationType = 'Offesnive'
elif(prediction[0]==2):
        classifciationType = 'Neither'


print(f"The prediction for the sentence is: {classifciationType}")

0    good
dtype: object
The prediction for the sentence is: Offesnive


# Web Scrapping

In [9]:
import pandas as pd
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter
scrapper=sntwitter.TwitterSearchScraper("#battle_bots")
for tweet in scrapper.get_items():
    break


Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22%23battle_bots%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everywhere_api

ScraperException: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22%23battle_bots%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts

# Descion tree

In [11]:
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline

# Load dataset
dataset = pd.read_csv("labeled_data.csv")

# Simple data cleaning process
def clean_text(text):
    text = text.lower()  # Lowercase text
    text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    return text

# Apply cleaning function to the 'tweet' column
dataset['processed_tweets'] = dataset['tweet'].apply(clean_text)

# Build features with TF-IDF on the cleaned tweets
tfidf_vectorizer = TfidfVectorizer(max_features=10000)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_tweets'], dataset['class'], test_size=0.2, random_state=42)

# Initialize the Decision Tree model
decision_tree_model = DecisionTreeClassifier(random_state=42)

# Create a pipeline that pipes the vectorization into the Decision Tree model
pipeline = make_pipeline(tfidf_vectorizer, decision_tree_model)

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8638289287875731
              precision    recall  f1-score   support

           0       0.31      0.25      0.28       290
           1       0.91      0.93      0.92      3832
           2       0.79      0.79      0.79       835

    accuracy                           0.86      4957
   macro avg       0.67      0.65      0.66      4957
weighted avg       0.86      0.86      0.86      4957



# KNN

In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
dataset = pd.read_csv("labeled_data.csv")
dataset

# Assuming your data is in a DataFrame called df
features = df[['count', 'hate_speech', 'offensive_language', 'neither', 'tweet']]
target = df['class']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Text vectorization using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')  # Adjust parameters as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['tweet'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['tweet'])

# Convert TF-IDF features to DataFrame with string column names
tfidf_columns = [f'tfidf_{i}' for i in range(X_train_tfidf.shape[1])]
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_columns)
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf_columns)

# Combine TF-IDF features with other numerical features
X_train_combined = pd.concat([X_train[['count', 'hate_speech', 'offensive_language', 'neither']].reset_index(drop=True),
X_train_tfidf_df], axis=1)

X_test_combined = pd.concat([X_test[['count', 'hate_speech', 'offensive_language', 'neither']].reset_index(drop=True),
X_test_tfidf_df], axis=1)

# Scale combined features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_test_scaled = scaler.transform(X_test_combined)

# Train the KNN model
k = 3  # Choose the value of k (number of neighbors)
knn_classifier = KNeighborsClassifier(n_neighbors=k)
knn_classifier.fit(X_train_scaled, y_train)

# Make predictions
y_pred = knn_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

classification_rep = classification_report(y_test, y_pred)
print(f'Classification Report:\n{classification_rep}')

def predict_offensiveness(new_input, vectorizer, scaler, classifier):
    # Text vectorization using the same TF-IDF vectorizer
    new_input_tfidf = vectorizer.transform([new_input])
    new_input_tfidf_df = pd.DataFrame(new_input_tfidf.toarray(), columns=tfidf_columns)

    # Combine TF-IDF features with other numerical features
    new_input_combined = pd.concat([pd.DataFrame([[0, 0, 0, 0]], columns=['count', 'hate_speech', 'offensive_language', 'neither']),
                                    new_input_tfidf_df], axis=1)

    # Scale combined features using the same scaler
    new_input_scaled = scaler.transform(new_input_combined)

    # Make prediction using the trained KNN model
    prediction = classifier.predict(new_input_scaled)
    
    return prediction[0]

# Test the function with a new input
new_word = " this is such a nice day"
prediction_result = predict_offensiveness(new_word, tfidf_vectorizer, scaler, knn_classifier)
final_result=''
print(prediction_result)
if prediction_result==1:
    final_result= 'bad'
elif prediction_result==2:
    final_result='not bad'

print(f'The predicted class for "{new_word}" is: {final_result}')

# my model consider such is bad word here

Accuracy: 0.80068589872907
Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.24      0.29       290
           1       0.84      0.94      0.88      3832
           2       0.66      0.37      0.47       835

    accuracy                           0.80      4957
   macro avg       0.62      0.52      0.55      4957
weighted avg       0.78      0.80      0.78      4957

1
The predicted class for " this is such a nice day" is: bad
